In [ ]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn, get_data, init_distributed_device
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda:0')

In [ ]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/pretrain_cvit.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args.finetune = False
args

In [ ]:
device = init_distributed_device(args)
cfg, _ = setup(args)

In [ ]:
cfg.DATASET['train_transforms']['normalize'] = True
cfg.DATASET['train_transforms']['standardize'] = False

In [ ]:
data = get_data(cfg)
data

In [ ]:
sample = iter(data['test'].dataloader).__next__()

In [ ]:
sample['image'].shape

In [ ]:
model = construct_mae(cfg.MODEL)

In [ ]:
device = 'cuda:0'
device_ids = [0, 1, 2, 3]

In [ ]:
model = model.to(device)
model = torch.nn.DataParallel(model, device_ids=device_ids)

In [ ]:
loss = get_loss_list(cfg.LOSS)

In [ ]:
loss

In [ ]:
device = torch.device(device)

In [ ]:
# image = sample['image'].to(device, non_blocking=True)
# radar = sample['radar'].to(device, non_blocking=True)
image = sample['image']
radar = sample['radar']

In [ ]:
# model_out = model(image, radar)
# logit_scale = model_out.get("logit_scale").mean()
# model_out['logit_scale'] = logit_scale

In [ ]:
# points = [[20, 193], [35, 180], [55, 150], [130, 180], [95, 140], [30, 25], [80, 30]]
points = [[45, 10], [45, 20]]
cross_length = 5
img_id = 100
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
color = ['r', 'g', 'b', 'cyan', 'orange', 'purple', 'brown']
for c, point in enumerate(points):
    curve = image[img_id, :, point[0], point[1]].detach().cpu().numpy()
    curve = np.delete(curve, 10)
    plt.plot(np.arange(12), curve, color[c])
vis_img = image[img_id, [3, 2, 1], :, :].detach().cpu().numpy().transpose(1, 2, 0)
# normalize each channel
vis_img = (vis_img - vis_img.min()) / (vis_img.max() - vis_img.min())

plt.subplot(1, 2, 2)
plt.imshow(vis_img)
# draw a cross at the point
for c, point in enumerate(points):
    plt.plot([point[1] - cross_length, point[1] + cross_length], [point[0], point[0]],  color[c]) 
    # Draw vertical line of the cross
    plt.plot([point[1], point[1]], [point[0] - cross_length, point[0] + cross_length],  color[c]) 
# plt.axis('off')

In [ ]:
# points = [[20, 193], [35, 180], [55, 150], [130, 180], [95, 140], [30, 25], [80, 30]]
points = [[45, 10], [45, 20]]
cross_length = 5
img_id = 150
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
color = ['r', 'g', 'b', 'cyan', 'orange', 'purple', 'brown']
for c, point in enumerate(points):
    curve = radar[img_id, :, point[0], point[1]].detach().cpu().numpy()
    # curve = np.delete(curve, 10)
    plt.plot(np.arange(2), curve, color[c])
vis_img = radar[img_id, [0], :, :].detach().cpu().numpy().transpose(1, 2, 0)
# normalize each channel
vis_img = (vis_img - vis_img.min()) / (vis_img.max() - vis_img.min())

plt.subplot(1, 2, 2)
plt.imshow(vis_img)
# draw a cross at the point
for c, point in enumerate(points):
    plt.plot([point[1] - cross_length, point[1] + cross_length], [point[0], point[0]],  color[c]) 
    # Draw vertical line of the cross
    plt.plot([point[1], point[1]], [point[0] - cross_length, point[0] + cross_length],  color[c]) 
# plt.axis('off')

In [ ]:
from GeospatialFM.models.costum_layers import *
import torch

%load_ext autoreload
%autoreload 2

In [ ]:
sample = torch.randn(2, 13, 224, 224)

In [ ]:
patch_embedding = PatchEmbedPerChannel(in_chans=13)

In [ ]:
extra_tokens = dict()
extra_tokens["channels"] = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]])
out = patch_embedding(sample, extra_tokens)

In [ ]:
collpase_channels(out[0], 'mean').shape

In [ ]:
out[1].shape

In [ ]:
out[1]

In [ ]:
13*0.25

In [ ]:
x = sample
B, Cin, H, W = x.shape
# Note: The current number of channels (Cin) can be smaller or equal to in_chans
len_keep = int(Cin * 0.4)

noise = torch.rand(B, Cin, device=x.device)  # noise in [0, 1]
# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=1)

# keep the first subset
ids_keep = ids_shuffle[:, :len_keep]

In [ ]:
ids_keep.unsqueeze(-1).unsqueeze(-1).repeat(1, 5, 224, 224)

In [ ]:
ids_keep.shape
# sample corresponding channels from ce
new_ce = torch.gather(ce, 2, ids_keep.unsqueeze(1).repeat(1, ce.shape[1], 1))

In [ ]:
new_ce[0, :5, 0], ids_keep[0, 0], ce[0, :5, 2]

In [ ]:
new_ce[0, :5].shape

In [ ]:
ce[0, :5, 2].shape

In [ ]:
# gather the C dim of the batch of the images in shape B x C x H x W following the index of ids_keep
x_keep = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, *x.shape[-2:]))

In [ ]:
x_keep.shape

In [ ]:
# generate the binary mask: 0 is keep, 1 is remove
mask = torch.ones([B, Cin], device=x.device)
mask[:, :len_keep] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)
# Per batch channel sampling
        # Note this may be slow

In [ ]:
mask

In [ ]:
ids_keep

In [ ]:
embedding = torch.rand((2, 3, 10))

In [ ]:
embedding.shape

In [ ]:
in_chans = 13
embed_dim = 768
patch_size = 16
sample = torch.randn(320, 13, 224, 224)
proj = nn.Conv2d(
    in_chans,
    embed_dim*in_chans,
    kernel_size=(patch_size, patch_size),
    stride=(patch_size, patch_size),
    groups=in_chans,
)  # CHANGED

In [ ]:
proj2 = nn.Conv2d(
    in_chans,
    embed_dim,
    kernel_size=(patch_size, patch_size),
    stride=(patch_size, patch_size),
    bias=False
)  # CHANGED

In [ ]:
proj3 = nn.Conv3d(
    1,
    embed_dim*in_chans,
    kernel_size=(1, patch_size, patch_size),
    stride=(1, patch_size, patch_size),
    bias=False
)  # CHANGED

In [ ]:
[p for p in proj.parameters()]

In [ ]:
[p.numel() for p in proj2.parameters()]

In [ ]:
# print the number of parameters
w1 = sum(p.numel() for p in proj.parameters())
w2 = sum(p.numel() for p in proj2.parameters())
w3 = sum(p.numel() for p in proj3.parameters())
w1-w2, w3-w2

In [ ]:
w1 = proj.weight.flatten().shape[0]
w2 = proj2.weight.flatten().shape[0]
w3 = proj3.weight.flatten().shape[0]
w1-w2, w3-w2

In [ ]:
b1 = proj.bias.flatten().shape[0]
b2 = proj2.bias.flatten().shape[0]
b3 = proj3.bias.flatten().shape[0]
w1-w2, w3-w2

In [ ]:
proj.to('cuda:1')
proj2.to('cuda:1')
proj3.to('cuda:1')
sample = sample.to('cuda:1')

In [ ]:
# estimate the timw of the forward pass
import time
start = time.time()
out1 = proj(sample).cpu()
end = time.time()
print(end - start)

start = time.time()
out2 = proj2(sample).cpu()
end = time.time()
print(end - start)

start = time.time()     
out3 = proj3(sample.unsqueeze(1)).cpu()
end = time.time()
print(end - start)

In [ ]:
sample[:, torch.tensor([0, 1, 11])].shape

In [ ]:
out1.shape

In [ ]:
out3.shape

In [ ]:
import random

In [ ]:
random.uniform(0.25, 0.75)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
sample = torch.randn(2, 1, 224, 224)

In [ ]:
proj = nn.Conv3d(
    1,
    1024,
    kernel_size=(1, 4, 4),
    stride=(1, 4, 4),
)

In [ ]:
proj2 = nn.Conv3d(
    1024,
    768, 
    kernel_size=(1, 4, 4),
    stride=(1, 4, 4),
)

In [ ]:
proj3 = nn.Conv3d(
    768,
    768, 
    kernel_size=(1, 1, 1),
    stride=(1, 1, 1),
)

In [ ]:
proj3(proj2(proj(sample.unsqueeze(1)))).shape

In [ ]:
embed_dim = 768
proj = nn.Conv3d(
        embed_dim*2,
        embed_dim,
        kernel_size=(1, 1, 1),
        stride=(1, 1, 1),
    )
w = sum(p.numel() for p in proj.parameters())
w

In [ ]:
2*embed_dim**2

In [ ]:
import os
import pickle

In [ ]:
cache_path = 'results/cache/BigEarthNet_vit_base_patch16_224_mae_base_finetune_OPTICAL/'

In [ ]:
def load_features(cache_path, split):
    # list all files in cache_path containing split
    cache_files = [f for f in os.listdir(cache_path) if split in f and f.endswith('.pkl')]
    cache_files.sort()
    features = dict(label=[], img_feature=[])
    for f_name in cache_files:
        print(f_name)
        with open(os.path.join(cache_path, f_name), 'rb') as f:
            chunk_features = pickle.load(f)
            print(chunk_features)
            features['label'].append(chunk_features['label'])
            features['img_feature'].append(chunk_features['img_feature'])
    print(features['label'])
    features['label'] = torch.cat(features['label'], dim=0)
    features['img_feature'] = torch.cat(features['img_feature'], dim=0)
    return features

In [ ]:
load_features(cache_path, 'test')

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class ChannelINN(nn.Module):
    def __init__(self, n_nodes, out_channels, kernel_size, stride, padding=0):
        super(ChannelINN, self).__init__()
        self.conv = nn.Conv3d(1, n_nodes*out_channels, kernel_size, stride, padding)
        self.n_nodes = n_nodes
        self.out_channels = out_channels
        self._x = torch.linspace(0, 1, n_nodes)
    
    # def _interpolate(self, x, idx):
    def _intergral(self, x, idx):
        pass
        # x in shape B x n_nodes x C_out x H x W
        
 
    def forward(self, x, x_idx):
        # x in shape B x 1 x C x H x W
        assert x_idx.shape[0] == x.shape[2]
        x = self.conv(x) # B x n_nodes*out_channels C x H x W
        B, _, C, H, W = x.shape
        x = x.view(B, self.n_nodes, self.out_channels, C, H, W)
        x_ = torch.zeros(B, self.out_channels, C, H, W, device=x.device)
        for i in range(C):
            idx = x_idx[i]
            fx = x[:, :, :, i, :, :]
            x_[:, :, i, :, :] = self._interpolate(fx, idx)

        return x

In [ ]:
model = ChannelINN(20, 768, (1, 16, 16), (1, 16, 16))
sample = torch.randn(2, 13, 224, 224)

In [ ]:
model(sample.unsqueeze(1), torch.arange(13)).shape

In [2]:
import torchgeo.datamodules as tgdm

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dm = getattr(tgdm, 'EuroSATDataModule')

In [4]:
dm.mean

tensor([1354.4055, 1118.2440, 1042.9298,  947.6262, 1199.4729, 1999.7909,
        2369.2229, 2296.8262,  732.0834,   12.1133, 1819.0103, 1118.9240,
        2594.1409])

In [8]:
dm.std

tensor([ 245.7176,  333.0078,  395.0925,  593.7505,  566.4170,  861.1840,
        1086.6313, 1117.9817,  404.9198,    4.7758, 1002.5877,  761.3032,
        1231.5858])

In [10]:
dm = tgdm.So2SatDataModule

In [11]:
dm.mean, dm.std

(tensor(0), tensor(255))

In [12]:
dm = tgdm.BigEarthNetDataModule

In [13]:
dm.mean, dm.std

(tensor(0), tensor(255))

In [14]:
kwargs = {'bands': ['S2_B02', 'S2_B03', 'S2_B04', 'S2_B05', 'S2_B06', 'S2_B07', 'S2_B08', 'S2_B8A', 'S2_B11', 'S2_B12'], 'version': '3_culture_10', 'pad_s2': True, 'root': './data/geospatial/So2Sat'}

In [15]:
dm = tgdm.So2SatDataModule(**kwargs)

In [16]:
dm.mean, dm.std

(tensor(0), tensor(255))

In [1]:
import torch

In [3]:
torch.empty(1).uniform_(float(-30), float(30)).item()

-16.128097534179688

In [6]:
torch.random.choice([0, 1]).item()

AttributeError: module 'torch.random' has no attribute 'choice'

In [14]:
import random
degrees = [0, 90, 180, 270]
torch.tensor(random.choice(degrees), dtype=torch.float32)

tensor(270.)